In [2]:
import pandas as pd
import os
import math
import scipy.stats as st
import scipy.spatial.distance as sd
import sklearn.metrics as mt
import numpy as np


### Load 'procedure' dataset

In [3]:
#read procedure df


path='BIRAFFE2-procedure'
subjects_proc = os.listdir(path)
subjects_proc =[x for x in subjects_proc if x.endswith(".csv")]

df_procedure=pd.DataFrame()

for proc in subjects_proc:
    sub_name=proc.split('.csv')[0]
    df = pd.read_csv("{}/{}.csv".format(path,sub_name),delimiter=';',low_memory = False)
    
    sub_name=sub_name.split('-')[0]
    df['SUB-NAME'] = df.apply(lambda row: sub_name, axis=1)
    df = df[['SUB-NAME','TIMESTAMP','ID','COND','IADS-ID','IAPS-ID','ANS-VALENCE','ANS-AROUSAL','ANS-TIME','EVENT']]
    frames=[df_procedure,df]
    df_procedure=pd.concat(frames)
    
affective_condition_lst=['P+','P-','S+','S-','PS+','PS-','P0','S0','PS0']

df_procedure= df_procedure[df_procedure['COND'].isin(affective_condition_lst)]

df_procedure=df_procedure.reset_index()
df_procedure['STIMULI'] = df_procedure.apply(lambda row:'_'.join([str(df_procedure.loc[row.name,'SUB-NAME']),str(df_procedure.loc[row.name,'IADS-ID']),str(df_procedure.loc[row.name,'IAPS-ID'])]) , axis=1)
del df_procedure['index']

In [36]:
df_procedure

,SUB-NAME,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,STIMULI
0,SUB103,1.579508e+09,103.0,PS-,711,6315,2.897,6.591,3.133,NaN,SUB103_711_6315
1,SUB103,1.579508e+09,103.0,S-,244,None,2.291,5.625,5.727,NaN,SUB103_244_None
2,SUB103,1.579508e+09,103.0,PS0,246,2312,2.705,4.337,3.350,NaN,SUB103_246_2312
3,SUB103,1.579508e+09,103.0,S-,292,None,2.396,5.569,5.048,NaN,SUB103_292_None
4,SUB103,1.579508e+09,103.0,PS-,709,3170,3.414,6.875,4.808,NaN,SUB103_709_3170
...,...,...,...,...,...,...,...,...,...,...,...
11210,SUB997,1.580231e+09,997.0,P0,None,1301,7.295,6.474,1.317,NaN,SUB997_None_1301
11211,SUB997,1.580231e+09,997.0,S+,220,None,2.977,5.000,1.650,NaN,SUB997_220_None
11212,SUB997,1.580231e+09,997.0,PS+,415,8380,8.290,2.672,5.433,NaN,SUB997_415_8380
11213,SUB997,1.580231e+09,997.0,PS0,425,2635,5.000,1.000,2.250,NaN,SUB997_425_2635


### Load dataset with personalities of subjects

In [54]:
#read procedure df


path='BIRAFFE2-metadata.csv'
df_personality = pd.read_csv(path,delimiter=';')
cols_to_keep=['ID','OPENNESS','CONSCIENTIOUSNESS','EXTRAVERSION','AGREEABLENESS','NEUROTICISM']
df_personality=df_personality.drop(df_personality.columns.difference(cols_to_keep), axis=1)
df_personality.rename({'ID': 'SUB-NAME'}, axis=1, inplace=True)
df_personality['SUB-NAME'] = df_personality['SUB-NAME'].apply(lambda x: 'SUB{}'.format(x))

In [55]:
df_personality

,SUB-NAME,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
0,SUB103,6,6,2,3,4
1,SUB119,6,7,6,2,2
2,SUB120,1,2,2,2,2
3,SUB136,7,3,8,5,10
4,SUB141,3,1,7,5,4
...,...,...,...,...,...,...
98,SUB948,7,10,1,3,7
99,SUB959,8,7,2,8,4
100,SUB969,3,8,6,1,5
101,SUB977,6,7,2,7,8


### load 'photo-subjects' dataset

In [37]:
path='FACE-AFTER-CSV'
subjects = os.listdir(path)
subjects =[x for x in subjects if x.endswith(".csv")]

df_all_subs=pd.DataFrame()

for sub in subjects:
    sub_name=sub.split('.csv')[0]
    df = pd.read_csv("{}/{}.csv".format(path,sub_name),delimiter=';')
    
    sub_name=sub_name.split('-')[0]
    df['SUB-NAME'] = df.apply(lambda row: sub_name, axis=1)
    df = df[['SUB-NAME','GAME-TIMESTAMP','FRAME-NUMBER', 'IADS-ID', 'IAPS-ID', 'ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']]
    frames=[df_all_subs,df]
    df_all_subs=pd.concat(frames)
    
def is_recognized(row):
    if math.isnan(row[5]):
        return 0
    else:
        return 1
    
#exclude rows where emotions not recognized.    
df_all_subs['FACE_RECOGNIZED'] = df_all_subs.apply(lambda row: is_recognized(row), axis=1) 

grouped_after=df_all_subs.groupby('FACE_RECOGNIZED')
not_recognized_df=grouped_after.get_group(0)
recognized_df=grouped_after.get_group(1)

recognized_df=recognized_df.reset_index()

recognized_df['STIMULI'] = recognized_df.apply(lambda row:'_'.join([str(recognized_df.loc[row.name,'SUB-NAME']),str(recognized_df.loc[row.name,'IADS-ID']),str(recognized_df.loc[row.name,'IAPS-ID'])]) , axis=1)

#exclude game stimuli
affective_df= recognized_df[~recognized_df['GAME-TIMESTAMP'].notna()]
del affective_df['index']

In [38]:
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI
0,SUB103,NaN,0,102,None,0.0,0.001,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None
1,SUB103,NaN,105,102,None,0.0,0.001,0.0,0.0,0.000,0.996,0.003,0.0,1,SUB103_102_None
2,SUB103,NaN,120,102,None,0.0,0.000,0.0,0.0,0.000,0.992,0.008,0.0,1,SUB103_102_None
3,SUB103,NaN,135,102,None,0.0,0.000,0.0,0.0,0.000,0.995,0.005,0.0,1,SUB103_102_None
4,SUB103,NaN,15,102,None,0.0,0.000,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528680,SUB997,NaN,45,None,9594,0.0,0.000,0.0,0.0,0.002,0.996,0.002,0.0,1,SUB997_None_9594
528681,SUB997,NaN,60,None,9594,0.0,0.001,0.0,0.0,0.002,0.995,0.002,0.0,1,SUB997_None_9594
528682,SUB997,NaN,75,None,9594,0.0,0.002,0.0,0.0,0.003,0.991,0.004,0.0,1,SUB997_None_9594
528683,SUB997,NaN,90,None,9594,0.0,0.002,0.0,0.0,0.056,0.941,0.001,0.0,1,SUB997_None_9594


### Exclude rows where no stimuli was presented (instructions, tutorial etc.)

In [39]:
affective_stimuli_lst = df_procedure[['STIMULI']]
affective_stimuli_lst = [ x[-1] for x in df_procedure.values.tolist()]
affective_df=affective_df[affective_df['STIMULI'].isin(affective_stimuli_lst)]

### df with baseline for each stimuli. Also, cut prestim from main df

In [40]:
#look up prestim to use it as a baseline for stat distances.
baseline_df=affective_df[affective_df['FRAME-NUMBER']=='prestim']
#delete 'prestim' (baseline) rows so stat distances are not 0.
affective_df = affective_df[affective_df['FRAME-NUMBER'] != 'prestim']

In [78]:
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,1_highest emotion,2_highest emotion
0,SUB103,NaN,0,102,None,0.0,0.001,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000500,7.500000e-07,0.013693,0.000500,inf,0.026337,0.031631,0.001001,8.200701e-07,NEUTRAL,CONTEMPT/SADNESS
1,SUB103,NaN,105,102,None,0.0,0.001,0.0,0.0,0.000,0.996,0.003,0.0,1,SUB103_102_None,0.001000,3.250000e-06,0.017678,0.001000,inf,0.037260,0.044744,0.002004,4.448715e-06,NEUTRAL,SADNESS
2,SUB103,NaN,120,102,None,0.0,0.000,0.0,0.0,0.000,0.992,0.008,0.0,1,SUB103_102_None,0.002000,1.600000e-05,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,NEUTRAL,SADNESS
3,SUB103,NaN,135,102,None,0.0,0.000,0.0,0.0,0.000,0.995,0.005,0.0,1,SUB103_102_None,0.001250,6.250000e-06,0.017678,0.001250,inf,0.041665,0.050031,0.002506,1.238582e-05,NEUTRAL,SADNESS
4,SUB103,NaN,15,102,None,0.0,0.000,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000375,6.250000e-07,0.009682,0.000375,inf,0.018629,0.022372,0.001001,4.919014e-07,NEUTRAL,SADNESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528679,SUB997,NaN,345,None,9594,0.0,0.001,0.0,0.0,0.001,0.997,0.001,0.0,1,SUB997_None_9594,0.000750,1.500000e-06,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,NEUTRAL,CONTEMPT/HAPPINESS/SADNESS
528680,SUB997,NaN,45,None,9594,0.0,0.000,0.0,0.0,0.002,0.996,0.002,0.0,1,SUB997_None_9594,0.001000,3.000000e-06,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,NEUTRAL,HAPPINESS/SADNESS
528681,SUB997,NaN,60,None,9594,0.0,0.001,0.0,0.0,0.002,0.995,0.002,0.0,1,SUB997_None_9594,0.001250,4.250000e-06,0.023717,0.001250,inf,0.041665,0.050031,0.002506,3.137784e-06,NEUTRAL,HAPPINESS/SADNESS
528682,SUB997,NaN,75,None,9594,0.0,0.002,0.0,0.0,0.003,0.991,0.004,0.0,1,SUB997_None_9594,0.002250,1.375000e-05,0.031623,0.002250,inf,0.055940,0.067158,0.004520,1.016711e-05,NEUTRAL,SADNESS


### Add columns with stat distances

In [42]:
#stat distances to apply:


def hellinger(p, q):
    return math.sqrt(sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p, q) ]) / 2)

def bhattacharyya_distance(repr1, repr2):
#Calculates Bhattacharyya distance (https://en.wikipedia.org/wiki/Bhattacharyya_distance)."""
    sim = - np.log(np.sum([np.sqrt(p*q) for (p, q) in zip(repr1, repr2)]))
    assert not np.isnan(sim), 'Error: Similarity is nan.'
    if np.isinf(sim):
        # the similarity is -inf if no term in the review is in the vocabulary
        return 0
    return sim 

def get_entropy(f, s, mini=0.00000000001):
    # but entropy is not a metric!
    # replacing

    f[f == 0] = mini
    s[s == 0] = mini
    return st.entropy(f, s)


statistical_distances = ["mabse", "msqrte", "mse", "energy", "wasserstein",
                        "jensenshannon", "correlation",  ]



#in  case we theres no baseline for given stimulus, we can assume that baseline was 100% neutral expression. (???)
baseline_lst=[0,0,0,0,0,1.0,0,0]
functions=[(mt.mean_absolute_error,'mean_absolute_error'),(mt.mean_squared_error,'mean_squared_error'),
           (st.energy_distance,'energy'),(st.wasserstein_distance,'wasserstein'),
           (get_entropy,'relative entropy'),(sd.jensenshannon,'jensenshannon'),
           (hellinger,'hellinger'),(bhattacharyya_distance,'bhattacharyya_distance'),
           (sd.correlation,'correlation')
          ]

for func in functions:
    try:
        affective_df[func[1]] = affective_df.apply(lambda row: func[0](
                 row[['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist(),
                 baseline_df.loc[baseline_df['STIMULI'] == row['STIMULI']][['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist()[0]),
                 axis=1)
    except:
        affective_df[func[1]] = affective_df.apply(lambda row: func[0](
             row[['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist(),
             baseline_lst),
             axis=1)

C:\Users\bartk\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


In [27]:
#affective_df.dropna(axis=0,inplace=False)

In [43]:
pd.set_option('display.max_columns', None)
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation
0,SUB103,NaN,0,102,None,0.0,0.001,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000500,7.500000e-07,0.013693,0.000500,inf,0.026337,0.031631,0.001001,8.200701e-07
1,SUB103,NaN,105,102,None,0.0,0.001,0.0,0.0,0.000,0.996,0.003,0.0,1,SUB103_102_None,0.001000,3.250000e-06,0.017678,0.001000,inf,0.037260,0.044744,0.002004,4.448715e-06
2,SUB103,NaN,120,102,None,0.0,0.000,0.0,0.0,0.000,0.992,0.008,0.0,1,SUB103_102_None,0.002000,1.600000e-05,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05
3,SUB103,NaN,135,102,None,0.0,0.000,0.0,0.0,0.000,0.995,0.005,0.0,1,SUB103_102_None,0.001250,6.250000e-06,0.017678,0.001250,inf,0.041665,0.050031,0.002506,1.238582e-05
4,SUB103,NaN,15,102,None,0.0,0.000,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000375,6.250000e-07,0.009682,0.000375,inf,0.018629,0.022372,0.001001,4.919014e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528679,SUB997,NaN,345,None,9594,0.0,0.001,0.0,0.0,0.001,0.997,0.001,0.0,1,SUB997_None_9594,0.000750,1.500000e-06,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07
528680,SUB997,NaN,45,None,9594,0.0,0.000,0.0,0.0,0.002,0.996,0.002,0.0,1,SUB997_None_9594,0.001000,3.000000e-06,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06
528681,SUB997,NaN,60,None,9594,0.0,0.001,0.0,0.0,0.002,0.995,0.002,0.0,1,SUB997_None_9594,0.001250,4.250000e-06,0.023717,0.001250,inf,0.041665,0.050031,0.002506,3.137784e-06
528682,SUB997,NaN,75,None,9594,0.0,0.002,0.0,0.0,0.003,0.991,0.004,0.0,1,SUB997_None_9594,0.002250,1.375000e-05,0.031623,0.002250,inf,0.055940,0.067158,0.004520,1.016711e-05


### Add columns with first and second most recognized emotion

In [79]:
def emotion_val(row,top_no):
    #dict_emotions={0:'ANGER', 1:'CONTEMPT', 2:'DISGUST', 3:'FEAR', 4:'HAPPINESS', 5:'NEUTRAL', 6:'SADNESS', 7:'SURPRISE'}
    emotions=('ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS', 'SURPRISE')
    row=row.values.tolist()[5:12]
    row=zip(emotions,row)
    emotion_sorted=sorted(row,key=lambda x: x[1],reverse=True)
    
    result_1 = []
    result_2 = []
    for x,y in emotion_sorted:
        if y == emotion_sorted[0][1]:
            result_1.append(x)
        if y == emotion_sorted[1][1]:
            result_2.append(x)
    
    if emotion_sorted[top_no-1] == 0.0:
        return ['NONE']
    if top_no==1:
        return result_1
    if top_no==2:
        return result_2

affective_df['1_highest emotion'] = affective_df.apply(lambda row: emotion_val(row,1), axis=1)
affective_df['2_highest emotion'] = affective_df.apply(lambda row: emotion_val(row,2), axis=1)

In [82]:
full_df = affective_df.merge(df_procedure[['STIMULI',"COND", "ANS-VALENCE", "ANS-AROUSAL"]], on="STIMULI")

In [99]:
df_personality

,SUB_NAME,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
0,SUB103,6,6,2,3,4
1,SUB119,6,7,6,2,2
2,SUB120,1,2,2,2,2
3,SUB136,7,3,8,5,10
4,SUB141,3,1,7,5,4
...,...,...,...,...,...,...
98,SUB948,7,10,1,3,7
99,SUB959,8,7,2,8,4
100,SUB969,3,8,6,1,5
101,SUB977,6,7,2,7,8


In [101]:
#full_df.rename({'SUB-NAME': 'SUB_NAME'},axis=1,inplace=True)
full_df = full_df.merge(df_personality[['SUB_NAME','OPENNESS','CONSCIENTIOUSNESS','NEUROTICISM','AGREEABLENESS','EXTRAVERSION']], on="SUB_NAME")

In [102]:
full_df

,SUB_NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,1_highest emotion,2_highest emotion,COND,ANS-VALENCE,ANS-AROUSAL,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
0,SUB103,NaN,0,102,None,0.0,0.001,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000500,7.500000e-07,0.013693,0.000500,inf,0.026337,0.031631,0.001001,8.200701e-07,[NEUTRAL],"[CONTEMPT, SADNESS]",S0,6.582,5.00,6,6,2,3,4
1,SUB103,NaN,105,102,None,0.0,0.001,0.0,0.0,0.000,0.996,0.003,0.0,1,SUB103_102_None,0.001000,3.250000e-06,0.017678,0.001000,inf,0.037260,0.044744,0.002004,4.448715e-06,[NEUTRAL],[SADNESS],S0,6.582,5.00,6,6,2,3,4
2,SUB103,NaN,120,102,None,0.0,0.000,0.0,0.0,0.000,0.992,0.008,0.0,1,SUB103_102_None,0.002000,1.600000e-05,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00,6,6,2,3,4
3,SUB103,NaN,135,102,None,0.0,0.000,0.0,0.0,0.000,0.995,0.005,0.0,1,SUB103_102_None,0.001250,6.250000e-06,0.017678,0.001250,inf,0.041665,0.050031,0.002506,1.238582e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00,6,6,2,3,4
4,SUB103,NaN,15,102,None,0.0,0.000,0.0,0.0,0.000,0.998,0.001,0.0,1,SUB103_102_None,0.000375,6.250000e-07,0.009682,0.000375,inf,0.018629,0.022372,0.001001,4.919014e-07,[NEUTRAL],[SADNESS],S0,6.582,5.00,6,6,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256975,SUB997,NaN,345,None,9594,0.0,0.001,0.0,0.0,0.001,0.997,0.001,0.0,1,SUB997_None_9594,0.000750,1.500000e-06,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,[NEUTRAL],"[CONTEMPT, HAPPINESS, SADNESS]",P0,5.052,6.08,1,3,7,4,1
256976,SUB997,NaN,45,None,9594,0.0,0.000,0.0,0.0,0.002,0.996,0.002,0.0,1,SUB997_None_9594,0.001000,3.000000e-06,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,[NEUTRAL],"[HAPPINESS, SADNESS]",P0,5.052,6.08,1,3,7,4,1
256977,SUB997,NaN,60,None,9594,0.0,0.001,0.0,0.0,0.002,0.995,0.002,0.0,1,SUB997_None_9594,0.001250,4.250000e-06,0.023717,0.001250,inf,0.041665,0.050031,0.002506,3.137784e-06,[NEUTRAL],"[HAPPINESS, SADNESS]",P0,5.052,6.08,1,3,7,4,1
256978,SUB997,NaN,75,None,9594,0.0,0.002,0.0,0.0,0.003,0.991,0.004,0.0,1,SUB997_None_9594,0.002250,1.375000e-05,0.031623,0.002250,inf,0.055940,0.067158,0.004520,1.016711e-05,[NEUTRAL],[SADNESS],P0,5.052,6.08,1,3,7,4,1


In [103]:
del full_df['GAME-TIMESTAMP']

In [104]:
del full_df['FACE_RECOGNIZED']

In [105]:
full_df.sort_values(by=['STIMULI', 'NEUTRAL'],inplace=True)

In [106]:
full_df=full_df[['SUB_NAME','STIMULI','FRAME-NUMBER','IADS-ID','IAPS-ID','ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE','1_highest emotion','2_highest emotion','mean_absolute_error','mean_squared_error','energy','wasserstein','relative entropy','jensenshannon','hellinger','bhattacharyya_distance','correlation','COND','ANS-VALENCE','ANS-AROUSAL','OPENNESS','CONSCIENTIOUSNESS','NEUROTICISM','AGREEABLENESS','EXTRAVERSION']]

In [107]:
full_df

,SUB_NAME,STIMULI,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,1_highest emotion,2_highest emotion,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,COND,ANS-VALENCE,ANS-AROUSAL,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
16,SUB103,SUB103_102_None,300,102,None,0.001,0.207,0.0,0.0,0.000,0.789,0.003,0.0,[NEUTRAL],[CONTEMPT],0.052750,0.010922,0.116190,0.052750,inf,0.281714,0.334280,0.118494,3.436184e-02,S0,6.582,5.00,6,6,2,3,4
8,SUB103,SUB103_102_None,195,102,None,0.000,0.022,0.0,0.0,0.000,0.974,0.003,0.0,[NEUTRAL],[CONTEMPT],0.006375,0.000146,0.042205,0.006375,inf,0.093554,0.112186,0.013172,2.448778e-04,S0,6.582,5.00,6,6,2,3,4
10,SUB103,SUB103_102_None,225,102,None,0.000,0.001,0.0,0.0,0.000,0.985,0.014,0.0,[NEUTRAL],[SADNESS],0.003750,0.000053,0.031623,0.003750,inf,0.072298,0.086766,0.007557,9.750467e-05,S0,6.582,5.00,6,6,2,3,4
2,SUB103,SUB103_102_None,120,102,None,0.000,0.000,0.0,0.0,0.000,0.992,0.008,0.0,[NEUTRAL],[SADNESS],0.002000,0.000016,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,S0,6.582,5.00,6,6,2,3,4
9,SUB103,SUB103_102_None,210,102,None,0.000,0.000,0.0,0.0,0.000,0.993,0.006,0.0,[NEUTRAL],[SADNESS],0.001625,0.000011,0.020156,0.001625,inf,0.045673,0.054828,0.003512,1.791256e-05,S0,6.582,5.00,6,6,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256976,SUB997,SUB997_None_9594,45,None,9594,0.000,0.000,0.0,0.0,0.002,0.996,0.002,0.0,[NEUTRAL],"[HAPPINESS, SADNESS]",0.001000,0.000003,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,P0,5.052,6.08,1,3,7,4,1
256961,SUB997,SUB997_None_9594,150,None,9594,0.000,0.001,0.0,0.0,0.001,0.997,0.001,0.0,[NEUTRAL],"[CONTEMPT, HAPPINESS, SADNESS]",0.000750,0.000002,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,P0,5.052,6.08,1,3,7,4,1
256966,SUB997,SUB997_None_9594,225,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.000,0.0,[NEUTRAL],[HAPPINESS],0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,2.137071e-06,P0,5.052,6.08,1,3,7,4,1
256969,SUB997,SUB997_None_9594,270,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.001,0.0,[NEUTRAL],[HAPPINESS],0.000875,0.000002,0.020156,0.000875,inf,0.037241,0.044734,0.001502,2.137684e-06,P0,5.052,6.08,1,3,7,4,1


In [108]:
full_df.rename({
    
    'FRAME-NUMBER':'FRAME_NUMBER',
    'IADS-ID':'IADS_ID',
    'IAPS-ID':'IAPS_ID',
    'mean_absolute_error':'MEAN_ABS_ERR',
    'mean_squared_error':'MEAN_SQ_ERR',
    'energy':'ENERGY',
    'wasserstein':'WASSERSTEIN',
    'relative entropy':'RELATIVE_ENTROPY',
    'jensenshannon':'JENSENSHANNON',
    'hellinger':'HELLINGER',
    'bhattacharyya_distance':'BHATTACHARYYA_DIST',
    'correlation':'CORRELATION',
    '1_highest emotion':'1_HIGHEST_EMO',
    '2_highest emotion':'2_HIGHEST_EMO',
    'ANS-VALENCE':'ANS_VALENCE',
    'ANS-AROUSAL':'ANS_AROUSAL'
    
    }, axis=1, inplace=True)


In [111]:
full_df.rename({
    
    '1_HIGHEST_EMO':'FIRST_EMO',
    '2_HIGHEST_EMO':'SEC_EMO'

    
    }, axis=1, inplace=True)


In [112]:
full_df

,SUB_NAME,STIMULI,FRAME_NUMBER,IADS_ID,IAPS_ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FIRST_EMO,SEC_EMO,MEAN_ABS_ERR,MEAN_SQ_ERR,ENERGY,WASSERSTEIN,RELATIVE_ENTROPY,JENSENSHANNON,HELLINGER,BHATTACHARYYA_DIST,CORRELATION,COND,ANS_VALENCE,ANS_AROUSAL,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
16,SUB103,SUB103_102_None,300,102,None,0.001,0.207,0.0,0.0,0.000,0.789,0.003,0.0,[NEUTRAL],[CONTEMPT],0.052750,0.010922,0.116190,0.052750,inf,0.281714,0.334280,0.118494,3.436184e-02,S0,6.582,5.00,6,6,2,3,4
8,SUB103,SUB103_102_None,195,102,None,0.000,0.022,0.0,0.0,0.000,0.974,0.003,0.0,[NEUTRAL],[CONTEMPT],0.006375,0.000146,0.042205,0.006375,inf,0.093554,0.112186,0.013172,2.448778e-04,S0,6.582,5.00,6,6,2,3,4
10,SUB103,SUB103_102_None,225,102,None,0.000,0.001,0.0,0.0,0.000,0.985,0.014,0.0,[NEUTRAL],[SADNESS],0.003750,0.000053,0.031623,0.003750,inf,0.072298,0.086766,0.007557,9.750467e-05,S0,6.582,5.00,6,6,2,3,4
2,SUB103,SUB103_102_None,120,102,None,0.000,0.000,0.0,0.0,0.000,0.992,0.008,0.0,[NEUTRAL],[SADNESS],0.002000,0.000016,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,S0,6.582,5.00,6,6,2,3,4
9,SUB103,SUB103_102_None,210,102,None,0.000,0.000,0.0,0.0,0.000,0.993,0.006,0.0,[NEUTRAL],[SADNESS],0.001625,0.000011,0.020156,0.001625,inf,0.045673,0.054828,0.003512,1.791256e-05,S0,6.582,5.00,6,6,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256976,SUB997,SUB997_None_9594,45,None,9594,0.000,0.000,0.0,0.0,0.002,0.996,0.002,0.0,[NEUTRAL],"[HAPPINESS, SADNESS]",0.001000,0.000003,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,P0,5.052,6.08,1,3,7,4,1
256961,SUB997,SUB997_None_9594,150,None,9594,0.000,0.001,0.0,0.0,0.001,0.997,0.001,0.0,[NEUTRAL],"[CONTEMPT, HAPPINESS, SADNESS]",0.000750,0.000002,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,P0,5.052,6.08,1,3,7,4,1
256966,SUB997,SUB997_None_9594,225,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.000,0.0,[NEUTRAL],[HAPPINESS],0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,2.137071e-06,P0,5.052,6.08,1,3,7,4,1
256969,SUB997,SUB997_None_9594,270,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.001,0.0,[NEUTRAL],[HAPPINESS],0.000875,0.000002,0.020156,0.000875,inf,0.037241,0.044734,0.001502,2.137684e-06,P0,5.052,6.08,1,3,7,4,1


In [113]:
full_df.to_csv('emo_personality_procedure_df.csv', index = False)